# Analysing Coastal Change with Landsat

This notebook is a simplified, worked example of analysing multiple years of
Landsat data to locate the coastline, and its change over time.

The original implementation of this algorithm was achieved by Geoscience Australia
and is available at [dea-costlines](https://github.com/geoscienceAustralia/dea-coastlines).
There is a new version, which is more generic available at
[coastlines](https://github.com/auspatious/coastlines).

## Requirements

The algorithm uses two fundameltal datasets, first, optical Earth observation data,
and Landsat is ideal, as it goes back over forty years. And second, is a tidal
model, which is used to annotate scenes with a tide height, and to filter
those scenes to just those in the middle of the tide range, therefore establishing
an 'average' coastline.

## Configuration and setup

First, we import libraries and tools that we need to run the analysis.

In [ ]:
from pystac_client import Client
from odc.stac import load, configure_s3_access

from dask.distributed import Client as DaskClient

from dea_tools.coastal import pixel_tides
from dea_tools.spatial import points_on_line
from coastlines.utils import tide_cutoffs, extract_contours
from coastlines.vector import annual_movements, calculate_regressions
from ipyleaflet import basemaps

from pathlib import Path

import numpy as np

### Set up our environment

We're going to access Landsat data from USGS using the Element-84 STAC API.

The `configure_s3_access` function will set up the environment for the requester
pays bucket on S3, which USGS shares data from. And we use Dask to lazy-load
data and run the computation in parallel.

In [ ]:
# STAC Catalog URL
catalog = "https://earth-search.aws.element84.com/v1"

# Create a STAC Client
client = Client.open(catalog)

# Tide data and config
home = Path("~")
tide_data_location = f"{home}/tide_models"

# This line will fail if you don't have credentials configured
_ = configure_s3_access(cloud_defaults=True, requester_pays=True)

# Set up a dask client
dask_client = DaskClient(n_workers=4, threads_per_worker=8)
dask_client

### Set up a study location

Configure a spatial location and a time range.

In [ ]:
# Find a location you're interested in on Google Maps and copy the coordinates
# by right-clicking on the map and clicking the coordinates

# These coords are in the order Y then X, or Latitude then Longitude
coords = 12.293, 109.225  # Near Phuong Vinh Hoa 
buffer = 0.05
bbox = (coords[1] - buffer, coords[0] - buffer, coords[1] + buffer, coords[0] + buffer)
landsat_stretch = dict(vmin=7500, vmax=18000)

datetime = "2019/2024"

### Find and load data

Search for Landsat scenes from the STAC API.

Then lazy-load data using `odc-stac`.

In [ ]:
items = client.search(
    collections=["landsat-c2-l2"],
    bbox=bbox,
    datetime=datetime,
).item_collection()

print(f"Found {len(items)} items")

In [ ]:
data = load(
    items,
    bbox=bbox,
    collection="landsat-c2-l2",
    measurements=["red", "green", "blue", "nir08", "swir16", "qa_pixel"],
    groupby="solar_day",
    chunks={"x": 2048, "y": 2048},
)
data

### Preview data

This cell first selects the first four scenes using the `isel` or "index select" function.
And then plots that from an array, so that we get an RGB visualisation.

In [ ]:
subset = data[["red", "green", "blue"]].isel(time=range(0, 4))
subset.to_array().plot.imshow(col="time", col_wrap=2, size=6, **landsat_stretch)

## Data processing

Now we know we have some good data, we can start processing. First we need to mask
out clouds, so that they don't impact our results.

In [ ]:
# Bit flag mask for the QA_PIXEL band
# We need bits 3 and 4, which are the 4th and 5th bits from the right (0-indexed)
bitflags = 0b00011000
cloud_mask = (data.qa_pixel & bitflags) != 0

# Prepare a nodata mask
nodata = data.red == data.red.odc.nodata

# Combine the cloud mask and the nodata mask
mask = cloud_mask | nodata

# Apply the mask to the data
masked = data.where(~mask, other=np.nan)

In [ ]:
# Preview the masked data
masked_subset = masked[["red", "green", "blue"]].isel(time=range(0, 4))
masked_subset.to_array().plot.imshow(col="time", col_wrap=2, size=6, **landsat_stretch)

### Tide masking

This cell first filters out scenes that are wholy in the "extreme" tides, or those
outside the middle 50% of observations.

Next it does pixel-based masking of extreme tides, as within scenes, there are still
some regions affected by extreme tides.

In [ ]:
# Add tide height to the data
tides_hires, tides_lowres = pixel_tides(
    masked, resample=True, directory=tide_data_location, model="FES2022", dask_compute=True
)

# Determine tide cutoff
tide_cutoff_min, tide_cutoff_max = tide_cutoffs(data, tides_lowres, tide_centre=0.0)

tide_bool = (tides_hires >= tide_cutoff_min) & (tides_hires <= tide_cutoff_max)
data_filtered = data.sel(time=tide_bool.sum(dim=["x", "y"]) > 0)

# Apply mask, and load in corresponding tide masked data
data_tide_masked = data_filtered.where(tide_bool)

print(data_tide_masked)

### Identify land and water

We use a water index, here it's a combination of the normalised difference wetness index, NDWI,
and the modified version of that, MNDWI. We find the average of the two indices, which has been
found to be more robust to issues of noisy data over ocean in Landsat.

In [ ]:
# Create MNDWI index
data_tide_masked["mndwi"] = (data_tide_masked.green - data_tide_masked.swir16) / (data_tide_masked.green + data_tide_masked.swir16)
data_tide_masked["ndwi"] = (data_tide_masked.green - data_tide_masked.nir08) / (data_tide_masked.green + data_tide_masked.nir08)
data_tide_masked["combined"] = (data_tide_masked.mndwi + data_tide_masked.ndwi) / 2

# Group by year and calculate the median
combined_by_year = data_tide_masked.combined.groupby("time.year").median().to_dataset(name="combined").compute()
combined_by_year

In [ ]:
combined_by_year.combined.plot.imshow(col="year", col_wrap=2, size=6, cmap="RdBu", robust=True)

### Extract contours

Next we extract contour lines from the underlying land/water raster data.

In [ ]:
contour_gdf = extract_contours(combined_by_year)

contour_gdf.reset_index().explore(
    column="year",
    cmap="magma",
    style_kwds={"weight": 3},
    tiles=basemaps.Esri.WorldImagery
)

### Extract points

And from the contours, we extract points, and annotate them with change over time, so that
we can document how much the coastline has eroded or accreted.

In [ ]:
# Extract points at every 30 metres along the most recent shoreline
points_gdf = points_on_line(contour_gdf, index=2023, distance=30)

# Calculate annual movements based on the points from above
points_gdf = annual_movements(
    points_gdf, contours_gdf=contour_gdf, yearly_ds=combined_by_year, baseline_year=2023, water_index="combined"
)

# And regression lines
points_gdf = calculate_regressions(points_gdf=points_gdf)

## Visualisation

Finally, visualise the results together.

In [ ]:
# Add human-friendly label for plotting
points_gdf["Coastal change"] = points_gdf.apply(
    lambda x: f'<h4>This coastline has {"<b>retreated</b>" if x.rate_time < 0 else "<b>grown</b>"} '
    f"by</br><b>{x.rate_time:.2f} m (±{x.se_time:.1f}) per year</b> since "
    f"<b>{contour_gdf.index[0]}</b></h4>",
    axis=1,
)
points_gdf.loc[points_gdf.sig_time > 0.05, "Coastal change"] = f"<h4>No significant trend of retreat or growth)</h4>"

m = contour_gdf.reset_index().explore(
    column="year",
    cmap="inferno",
    tooltip=False,
    style_kwds={"opacity": 0.5},
    categorical=True,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="ESRI WorldImagery",
)

points_gdf.explore(
    m=m,
    column="rate_time",
    cmap="RdBu",
    markersize=5,
    tooltip="Coastal change",
)